# Testing Data Hypothesis

In [0]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import ToTensor,Resize

from PIL import Image
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import time
import sys
import csv
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.utils.data import Dataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image
import torch.nn.functional as func

from sklearn.metrics.ranking import roc_auc_score
import sklearn.metrics as metrics
import random

use_gpu = torch.cuda.is_available()

In [0]:
#Transformations
transformList = []
transformList.append(transforms.Resize((256,256)))
#transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))      
transformSequence=transforms.Compose(transformList)

### Testing with DenseNets

In [0]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda:0')

input_size = 196608
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        
        self.densenet = torchvision.models.densenet121(pretrained=True)
        self.densenet.classifier = nn.Sequential(
            nn.Linear(self.densenet.classifier.in_features, num_classes),
            nn.Sigmoid()
        )
        
    def forward(self,x):
        x = self.densenet(x)
        return x


In [0]:
device = torch.device("cuda")
model = Net(14)
model.load_state_dict(torch.load('densenet_5epochs.pt', map_location="cuda:0")) 
model = model.cuda()

In [0]:
transformSequence=transforms.Compose(transformList)

In [0]:
## Hypothesis 1: White color around the lungs increases the likelyhood of a pathology being present

In [0]:
class TestDataset(Dataset):
    def __init__(self, transform=None):
        image_files = []
        image_files.append('effusion (1).jpg')
        image_files.append('Lesion.jpg')
        image_files.append('Orginal.jpg')
        image_files.append('sample_data/Lesion 2.jpg')
        image_files.append('Bone Fracture 3.jpg')
        image_files.append('Edema Lvl 1.jpg')
        image_files.append('Edema Lvl 3.jpg')
        image_files.append('Lesion.jpg')
        image_files.append('Bone Fracture 2.jpg')
        image_files.append('Darker Version.jpg')
        image_files.append('Edema Lvl 2.jpg')
        image_files.append('Lesion 2.jpg')
        self.image_files = image_files
        self.transform = transform

    def __getitem__(self, index):
        location = self.image_files[index]
        image = Image.open(location).convert('RGB')
        study = location.rsplit('/', 1)[0]
        if self.transform is not None:
            image = self.transform(image)
        return image, study

    def __len__(self):
        return len(self.image_files)

In [0]:
#Transformations
transformList = []
transformList.append(transforms.Resize((256,256)))
#transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))      
transformSequence=transforms.Compose(transformList)

In [0]:
testDataset = TestDataset(transformSequence) 

In [0]:
test_data_loader = DataLoader(testDataset, batch_size= 2, shuffle = False)


In [0]:
results_df = pd.DataFrame(columns=['Cardiomegaly','Edema','Consolidation','Atelectasis','Pleural Effusion', 'Study'])

In [0]:
model.eval()
with torch.no_grad():
    #print(image_files)
    for image, location in test_data_loader:
        print(image.shape)
        image = image.cuda()
        pred_label = model(image)
        print(location)
        locationnp  = np.asarray(location)
        data = np.column_stack((pred_label.cpu().numpy()[:,[2,5,6,8,10]],locationnp))
        df = pd.DataFrame(data, columns=['Cardiomegaly','Edema','Consolidation','Atelectasis','Pleural Effusion', 'Study'])
        results_df = results_df.append(df)
        print(df)
       


torch.Size([2, 3, 256, 256])
('effusion (1).jpg', 'Lesion.jpg')
  Cardiomegaly        Edema Consolidation Atelectasis Pleural Effusion  \
0    0.1572724   0.16132286    0.17990588  0.37093908        0.8927877   
1  0.031204078  0.022872426   0.071938716  0.10460749      0.034354676   

              Study  
0  effusion (1).jpg  
1        Lesion.jpg  
torch.Size([2, 3, 256, 256])
('Orginal.jpg', 'sample_data')
  Cardiomegaly        Edema Consolidation  Atelectasis Pleural Effusion  \
0  0.022906184   0.02041955   0.022690155  0.074977286      0.020221872   
1  0.023046516  0.027376648   0.026098425    0.0838077      0.018793723   

         Study  
0  Orginal.jpg  
1  sample_data  


In [0]:
results_df

,Cardiomegaly,Edema,Consolidation,Atelectasis,Pleural Effusion,Study
0,0.1572724,0.16132286,0.17990588,0.37093908,0.8927877,effusion (1).jpg
1,0.031204078,0.022872426,0.071938716,0.10460749,0.034354676,Lesion.jpg
0,0.022906184,0.02041955,0.022690155,0.074977286,0.020221872,Orginal.jpg
1,0.1572724,0.16132286,0.17990588,0.37093908,0.8927877,effusion (1).jpg


In [0]:
results_df = results_df.groupby(by='Study').max()[['Cardiomegaly','Edema','Consolidation','Atelectasis','Pleural Effusion']].reset_index()
results_df